## Demand Estimation Exercise

In this assignment, you will learn how to simulate market data and estimate simple random coefficient demand models. The assignment should serve as an introduction to the structural estimation in heterogenous product demand models.

Let's first define the model.


In [1]:
import simulate
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt

In [2]:
market = simulate.Market_Data(10, 1000, 3, 10)
print(market)

Market with 10 firms and 1000 consumers over 10 time periods. 
 Firms sell differentiated product which have 3 product characteristics


In [12]:
mean_indirect_utilities = np.zeros((10*10, 1))


for t in range (0, 4, 1): 
    mean_indirect_utilities[t*10:10+t*10] = market.gen_mean_indirect_utilities(t)


print(mean_indirect_utilities)


[[22.07221752]
 [24.6856799 ]
 [26.82889965]
 [24.19338441]
 [13.87667439]
 [24.43346227]
 [19.04607282]
 [ 8.4802178 ]
 [20.85818766]
 [15.30031442]
 [20.93070506]
 [25.6478484 ]
 [27.2734481 ]
 [19.49763462]
 [16.50938527]
 [26.22160245]
 [19.48007604]
 [ 6.53300002]
 [22.5106498 ]
 [16.09158132]
 [21.89287356]
 [26.09239684]
 [22.57769831]
 [22.13034551]
 [18.29752546]
 [26.65560567]
 [17.53285825]
 [ 8.18546216]
 [23.3019167 ]
 [14.12852946]
 [22.337422  ]
 [21.39664705]
 [25.21040919]
 [23.91848569]
 [18.73152868]
 [24.70838788]
 [19.18532039]
 [ 8.97672905]
 [21.33886484]
 [13.71598249]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.        ]
 [ 0.     